# lab_results.ipynb
Author: UFO Software, LLC<br>
Created: Sunday, February 14, 2021 13:09<br>

License GPLv3+: GNU GPL version 3 or later <https://gnu.org/licenses/gpl.html> <br>
This is free software: you are free to change and redistribute it.<br>
There is NO WARRANTY, to the extent permitted by law.<br>
<br>
Reads in the LabResults_0.csv file from the [December 2020 WA State Traceability Data](https://lcb.app.box.com/s/fnku9nr22dhx04f6o646xv6ad6fswfy9?page=1).


In [2]:
import pandas as pd
import numpy as np
from pathlib import Path
import os

import seaborn as sns
import matplotlib.pyplot as plt
from pandas.plotting import scatter_matrix

# change the file path to match where your data is stored
file_path = Path('../dec-2020')
pd.set_option('display.max_columns', None)

In [3]:
def get_lab_results_df():
    # reduce the size of the dataframe's memory footprint by specifying data types
    # comment out columns you are not using to further decrease the memory footprint
    col_dtypes = {'global_id' : 'string',
                 'mme_id' : 'string',
                 'user_id' : 'string',
                 'external_id' : 'string',
                 'inventory_id' : 'string',
                 'status' : 'category',
                 'testing_status' : 'category',
                 'batch_id' : 'string',
                 'parent_lab_result_id' : 'string',
                 'og_parent_lab_result_id' : 'string',
                 'copied_from_lab_id' : 'string',
                 'lab_user_id' : 'string',
                 'type' : 'category',
                 'foreign_matter' : 'bool',
                 'moisture_content_percent' : 'float32', #if you are not using Dask change this to float16
                 'growth_regulators_ppm' : 'float32', #if you are not using Dask change this to float16
                 'cannabinoid_status' : 'category',
                 'cannabinoid_editor' : 'float32', #if you are not using Dask change this to float16
                 'cannabinoid_d9_thca_percent': 'float32',
                 'cannabinoid_d9_thca_mg_g' : 'float32', #if you are not using Dask change this to float16
                 'cannabinoid_d9_thc_percent' : 'float32', #if you are not using Dask change this to float16
                 'cannabinoid_d9_thc_mg_g' : 'float32', #if you are not using Dask change this to float16
                 'cannabinoid_d8_thc_percent' : 'float32', #if you are not using Dask change this to float16
                 'cannabinoid_d8_thc_mg_g' : 'float32', #if you are not using Dask change this to float16
                 'cannabinoid_cbd_percent' : 'float32', #if you are not using Dask change this to float16
                 'cannabinoid_cbd_mg_g' : 'float32', #if you are not using Dask change this to float16
                 'cannabinoid_cbda_percent' : 'float32', #if you are not using Dask change this to float16
                 'cannabinoid_cbda_mg_g' : 'float32', #if you are not using Dask change this to float16
                 'cannabinoid_cbdv_percent' : 'float32', #if you are not using Dask change this to float16
                 'cannabinoid_cbg_percent' : 'float32', #if you are not using Dask change this to float16
                 'cannabinoid_cbg_mg_g' : 'float32', #if you are not using Dask change this to float16
                 'terpenoid_pinene_percent' : 'float32', #if you are not using Dask change this to float16
                 'terpenoid_pinene_mg_g' : 'float32', #if you are not using Dask change this to float16
                 'microbial_status' : 'category',
                 'microbial_editor' : 'string',
                 'microbial_bile_tolerant_cfu_g' : 'float32', #if you are not using Dask change this to float16
                 'microbial_pathogenic_e_coli_cfu_g' : 'float32', #if you are not using Dask change this to float16
                 'microbial_salmonella_cfu_g' : 'float32', #if you are not using Dask change this to float16
                 'mycotoxin_status' : 'category',
                 'mycotoxin_editor' : 'string',
                 'mycotoxin_aflatoxins_ppb' : 'float32', #if you are not using Dask change this to float16
                 'mycotoxin_ochratoxin_ppb' : 'float32', #if you are not using Dask change this to float16
                 'metal_status' : 'category',
                 'metal_editor': 'string',
                 'metal_arsenic_ppm' : 'float32', #if you are not using Dask change this to float16
                 'metal_cadmium_ppm' : 'float32', #if you are not using Dask change this to float16
                 'metal_lead_ppm' : 'float32', #if you are not using Dask change this to float16
                 'metal_mercury_ppm' : 'float32', #if you are not using Dask change this to float16
                 'pesticide_status' : 'category',
                 'pesticide_editor' : 'string',
                 'pesticide_abamectin_ppm' : 'float32', #if you are not using Dask change this to float16
                 'pesticide_acequinocyl_ppm' : 'float32', #if you are not using Dask change this to float16
                 'pesticide_bifenazate_ppm' : 'float32', #if you are not using Dask change this to float16
                 'pesticide_cyfluthrin_ppm' : 'float32', #if you are not using Dask change this to float16
                 'pesticide_cypermethrin_ppm' : 'float32', #if you are not using Dask change this to float16
                 'pesticide_etoxazole_ppm' : 'float32', #if you are not using Dask change this to float16
                 'pesticide_flonicamid_ppm' : 'float32', #if you are not using Dask change this to float16
                 'pesticide_fludioxonil_ppm' : 'float32', #if you are not using Dask change this to float16
                 'pesticide_imidacloprid_ppm' : 'float32', #if you are not using Dask change this to float16
                 'pesticide_myclobutanil_ppm' : 'float32', #if you are not using Dask change this to float16
                 'pesticide_spinosad_ppm' : 'float32', #if you are not using Dask change this to float16
                 'pesticide_spirotetramet_ppm' : 'float32', #if you are not using Dask change this to float16
                 'pesticide_thiamethoxam_ppm' : 'float32', #if you are not using Dask change this to float16
                 'pesticide_trifloxystrobin_ppm' : 'float32', #if you are not using Dask change this to float16
                 'solvent_status' : 'category',
                 'solvent_editor' : 'string',
                 'solvent_butanes_ppm' : 'float32', #if you are not using Dask change this to float16
                 'solvent_heptane_ppm' : 'float32', #if you are not using Dask change this to float16
                 'solvent_propane_ppm' : 'float32', #if you are not using Dask change this to float16
                 'notes' : 'float32', #if you are not using Dask change this to float16
                 'thc_percent' : 'float32', #if you are not using Dask change this to float16
                 'intermediate_type' : 'category',
                 'moisture_content_water_activity_rate' : 'float32', #if you are not using Dask change this to float16
                 'solvent_acetone_ppm' : 'float32', #if you are not using Dask change this to float16
                 'solvent_benzene_ppm' : 'float32', #if you are not using Dask change this to float16
                 'solvent_cyclohexane_ppm' : 'float32', #if you are not using Dask change this to float16
                 'solvent_chloroform_ppm' : 'float32', #if you are not using Dask change this to float16
                 'solvent_dichloromethane_ppm' : 'float32', #if you are not using Dask change this to float16
                 'solvent_ethyl_acetate_ppm' : 'float32', #if you are not using Dask change this to float16
                 'solvent_hexanes_ppm' : 'float32', #if you are not using Dask change this to float16
                 'solvent_isopropanol_ppm' : 'float32', #if you are not using Dask change this to float16
                 'solvent_methanol_ppm' : 'float32', #if you are not using Dask change this to float16
                 'solvent_pentanes_ppm' : 'float32', #if you are not using Dask change this to float16
                 'solvent_toluene_ppm' : 'float32', #if you are not using Dask change this to float16
                 'solvent_xylene_ppm' : 'float32', #if you are not using Dask change this to float16
                 'pesticide_acephate_ppm' : 'float32', #if you are not using Dask change this to float16
                 'pesticide_acetamiprid_ppm' : 'float32', #if you are not using Dask change this to float16
                 'pesticide_aldicarb_ppm' : 'float32', #if you are not using Dask change this to float16
                 'pesticide_azoxystrobin_ppm' : 'float32', #if you are not using Dask change this to float16
                 'pesticide_bifenthrin_ppm' : 'float32', #if you are not using Dask change this to float16
                 'pesticide_boscalid_ppm' : 'float32', #if you are not using Dask change this to float16
                 'pesticide_carbaryl_ppm' : 'float32', #if you are not using Dask change this to float16
                 'pesticide_carbofuran_ppm' : 'float32', #if you are not using Dask change this to float16
                 'pesticide_chlorantraniliprole_ppm' : 'float32' #if you are not using Dask change this to float16
                }

    date_cols = ['created_at',
                 'deleted_at',
                 'updated_at',
                 'tested_at',
                 'received_at']

    # combine the column names to load only the columns you are using
    cols = list(col_dtypes.keys()) + date_cols

    lab_results_df = pd.read_csv(file_path / 'LabResults_0.csv', sep = '\t', encoding = 'utf-16', usecols = cols, dtype = col_dtypes, parse_dates = date_cols)
    # all the datasets in the WA data use global_id but it has different meaning for each dataset which makes the data difficult to understand and causes issues with Pandas when trying to perform operations on more than one dataframe.
    lab_results_df.rename(columns={'global_id':'lab_results_id'}, inplace=True)
 
    return lab_results_df

In [4]:
lab_results_df = get_lab_results_df()

In [5]:
lab_results_df.head()

,global_id,created_at,mme_id,user_id,external_id,deleted_at,inventory_id,updated_at,status,testing_status,batch_id,parent_lab_result_id,og_parent_lab_result_id,copied_from_lab_id,lab_user_id,tested_at,received_at,type,foreign_matter,moisture_content_percent,growth_regulators_ppm,cannabinoid_status,cannabinoid_editor,cannabinoid_d9_thca_percent,cannabinoid_d9_thca_mg_g,cannabinoid_d9_thc_percent,cannabinoid_d9_thc_mg_g,cannabinoid_d8_thc_percent,cannabinoid_d8_thc_mg_g,cannabinoid_cbd_percent,cannabinoid_cbd_mg_g,cannabinoid_cbda_percent,cannabinoid_cbda_mg_g,cannabinoid_cbdv_percent,cannabinoid_cbg_percent,cannabinoid_cbg_mg_g,terpenoid_pinene_percent,terpenoid_pinene_mg_g,microbial_status,microbial_editor,microbial_bile_tolerant_cfu_g,microbial_pathogenic_e_coli_cfu_g,microbial_salmonella_cfu_g,mycotoxin_status,mycotoxin_editor,mycotoxin_aflatoxins_ppb,mycotoxin_ochratoxin_ppb,metal_status,metal_editor,metal_arsenic_ppm,metal_cadmium_ppm,metal_lead_ppm,metal_mercury_ppm,pesticide_status,pesticide_editor,pesticide_abamectin_ppm,pesticide_acequinocyl_ppm,pesticide_bifenazate_ppm,pesticide_cyfluthrin_ppm,pesticide_cypermethrin_ppm,pesticide_etoxazole_ppm,pesticide_flonicamid_ppm,pesticide_fludioxonil_ppm,pesticide_imidacloprid_ppm,pesticide_myclobutanil_ppm,pesticide_spinosad_ppm,pesticide_spirotetramet_ppm,pesticide_thiamethoxam_ppm,pesticide_trifloxystrobin_ppm,solvent_status,solvent_editor,solvent_butanes_ppm,solvent_heptane_ppm,solvent_propane_ppm,notes,thc_percent,intermediate_type,moisture_content_water_activity_rate,solvent_acetone_ppm,solvent_benzene_ppm,solvent_cyclohexane_ppm,solvent_chloroform_ppm,solvent_dichloromethane_ppm,solvent_ethyl_acetate_ppm,solvent_hexanes_ppm,solvent_isopropanol_ppm,solvent_methanol_ppm,solvent_pentanes_ppm,solvent_toluene_ppm,solvent_xylene_ppm,pesticide_acephate_ppm,pesticide_acetamiprid_ppm,pesticide_aldicarb_ppm,pesticide_azoxystrobin_ppm,pesticide_bifenthrin_ppm,pesticide_boscalid_ppm,pesticide_carbaryl_ppm,pesticide_carbofuran_ppm,pesticide_chlorantraniliprole_ppm
0,<NA>,2018-01-31 22:34:19,WAWA1.MMCY,WAWA1.US29D,5275985106676286,NaT,WAJ412964.INCP,2018-01-31 22:34:19,not tested,completed,WAJ412964.BA1JS,<NA>,<NA>,<NA>,WAWA1.US29D,2016-04-22 02:30:00,1900-01-01,intermediate_product,False,0.0,0.0,completed,NaN,75.599998,NaN,6.80,NaN,0.0,NaN,2.9,NaN,0.00,NaN,NaN,0.0,NaN,NaN,NaN,completed,WAWA1.US1,0.0,0.0,NaN,completed,WAWA1.US1,0.0,0.0,NaN,<NA>,0.0,0.0,0.0,0.0,NaN,<NA>,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,completed,WAWA1.US1,0.0,0.0,0.0,NaN,76.000000,hydrocarbon_concentrate,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,WAJ412964.LR2,2018-01-31 22:36:20,WAWA1.MMCY,WAWA1.US29D,4129640000001061,NaT,WAJ412964.ING1,2018-01-31 22:36:20,passed,completed,WAJ412964.BA1LC,<NA>,<NA>,<NA>,WAWA1.US29D,2017-11-13 00:36:00,1900-01-01,harvest_materials,False,4.9,0.0,completed,NaN,19.770000,NaN,0.32,NaN,0.0,NaN,0.0,NaN,0.53,NaN,NaN,0.0,NaN,NaN,NaN,completed,WAWA1.US1,0.0,0.0,NaN,completed,WAWA1.US1,0.0,0.0,NaN,<NA>,0.0,0.0,0.0,0.0,NaN,<NA>,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,completed,WAWA1.US1,0.0,0.0,0.0,NaN,18.122999,flower_lots,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,<NA>,2018-01-31 22:39:17,WAWA1.MMCY,WAWA1.US29D,5275985106676286,NaT,WAJ412964.INCP,2018-01-31 22:39:17,not tested,completed,WAJ412964.BA1JS,<NA>,<NA>,<NA>,WAWA1.US29D,2016-04-22 02:30:00,1900-01-01,intermediate_product,False,0.0,0.0,completed,NaN,75.599998,NaN,6.80,NaN,0.0,NaN,2.9,NaN,0.00,NaN,NaN,0.0,NaN,NaN,NaN,completed,WAWA1.US1,0.0,0.0,NaN,completed,WAWA1.US1,0.0,0.0,NaN,<NA>,0.0,0.0,0.0,0.0,NaN,<NA>,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,completed,WAWA1.US1,0.0,0.0,0.0,NaN,76.000000,hydrocarbon_concentrate,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,<NA>,2018-01-31 22:44:04,WAWA1.MMCY,WAWA1.US29D,5275985106676286,NaT,WAJ412964.INCP,2018

In [6]:
lab_results_df.tail()

,global_id,created_at,mme_id,user_id,external_id,deleted_at,inventory_id,updated_at,status,testing_status,batch_id,parent_lab_result_id,og_parent_lab_result_id,copied_from_lab_id,lab_user_id,tested_at,received_at,type,foreign_matter,moisture_content_percent,growth_regulators_ppm,cannabinoid_status,cannabinoid_editor,cannabinoid_d9_thca_percent,cannabinoid_d9_thca_mg_g,cannabinoid_d9_thc_percent,cannabinoid_d9_thc_mg_g,cannabinoid_d8_thc_percent,cannabinoid_d8_thc_mg_g,cannabinoid_cbd_percent,cannabinoid_cbd_mg_g,cannabinoid_cbda_percent,cannabinoid_cbda_mg_g,cannabinoid_cbdv_percent,cannabinoid_cbg_percent,cannabinoid_cbg_mg_g,terpenoid_pinene_percent,terpenoid_pinene_mg_g,microbial_status,microbial_editor,microbial_bile_tolerant_cfu_g,microbial_pathogenic_e_coli_cfu_g,microbial_salmonella_cfu_g,mycotoxin_status,mycotoxin_editor,mycotoxin_aflatoxins_ppb,mycotoxin_ochratoxin_ppb,metal_status,metal_editor,metal_arsenic_ppm,metal_cadmium_ppm,metal_lead_ppm,metal_mercury_ppm,pesticide_status,pesticide_editor,pesticide_abamectin_ppm,pesticide_acequinocyl_ppm,pesticide_bifenazate_ppm,pesticide_cyfluthrin_ppm,pesticide_cypermethrin_ppm,pesticide_etoxazole_ppm,pesticide_flonicamid_ppm,pesticide_fludioxonil_ppm,pesticide_imidacloprid_ppm,pesticide_myclobutanil_ppm,pesticide_spinosad_ppm,pesticide_spirotetramet_ppm,pesticide_thiamethoxam_ppm,pesticide_trifloxystrobin_ppm,solvent_status,solvent_editor,solvent_butanes_ppm,solvent_heptane_ppm,solvent_propane_ppm,notes,thc_percent,intermediate_type,moisture_content_water_activity_rate,solvent_acetone_ppm,solvent_benzene_ppm,solvent_cyclohexane_ppm,solvent_chloroform_ppm,solvent_dichloromethane_ppm,solvent_ethyl_acetate_ppm,solvent_hexanes_ppm,solvent_isopropanol_ppm,solvent_methanol_ppm,solvent_pentanes_ppm,solvent_toluene_ppm,solvent_xylene_ppm,pesticide_acephate_ppm,pesticide_acetamiprid_ppm,pesticide_aldicarb_ppm,pesticide_azoxystrobin_ppm,pesticide_bifenthrin_ppm,pesticide_boscalid_ppm,pesticide_carbaryl_ppm,pesticide_carbofuran_ppm,pesticide_chlorantraniliprole_ppm
1993088,WAL12.LR16PX3,2021-01-06 09:31:22,WAWA1.MM1L9,WAWA1.US2F3,<NA>,NaT,WAL12.INSM2DX,2021-01-06 09:31:22,passed,completed,WAL12.BAMDJT6,<NA>,<NA>,<NA>,WAWA1.US2F3,2021-01-04 09:29:00,1900-01-01,harvest_materials,False,10.0,NaN,completed,3135.0,25.190001,NaN,0.30,NaN,NaN,NaN,0.21,NaN,0.06,NaN,NaN,NaN,NaN,NaN,NaN,completed,WAWA1.US2F3,0.0,0.0,0.0,completed,WAWA1.US2F3,0.0,0.0,NaN,<NA>,NaN,NaN,NaN,NaN,NaN,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<NA>,NaN,NaN,NaN,NaN,NaN,flower_lots,0.24,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1993089,WAL12.LR16PX4,2021-01-06 09:32:41,WAWA1.MM1L9,WAWA1.US2F3,<NA>,NaT,WAL12.INSM2DY,2021-01-06 09:32:41,passed,completed,WAL12.BAMDJT7,<NA>,<NA>,<NA>,WAWA1.US2F3,2021-01-04 09:31:00,1900-01-01,harvest_materials,False,10.8,NaN,completed,3135.0,25.070000,NaN,0.33,NaN,NaN,NaN,0.21,NaN,0.06,NaN,NaN,NaN,NaN,NaN,NaN,completed,WAWA1.US2F3,0.0,0.0,0.0,completed,WAWA1.US2F3,0.0,0.0,NaN,<NA>,NaN,NaN,NaN,NaN,NaN,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<NA>,NaN,NaN,NaN,NaN,NaN,flower_lots,0.23,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1993090,WAL12.LR16PX5,2021-01-06 09:33:26,WAWA1.MM1L9,WAWA1.US2F3,<NA>,NaT,WAL12.INSM2DZ,2021-01-06 09:33:26,passed,completed,WAL12.BAMDJT8,<NA>,<NA>,<NA>,WAWA1.US2F3,2021-01-04 09:32:00,1900-01-01,harvest_materials,False,10.8,NaN,completed,3135.0,22.389999,NaN,0.27,NaN,NaN,NaN,0.18,NaN,0.05,NaN,NaN,NaN,NaN,NaN,NaN,completed,WAWA1.US2F3,0.0,0.0,0.0,completed,WAWA1.US2F3,0.0,0.0,NaN,<NA>,NaN,NaN,NaN,NaN,NaN,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<NA>,NaN,NaN,NaN,NaN,NaN,flower_lots,0.24,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1993091,WAL12.LR16PX6,2021-01-06 09:34:44,WAWA1.MM1L9,WAWA1.US2F3,<NA>,NaT,WAL12.INSM2E0,2021-01-06 09:34:44,passed,completed,WAL12.BAMDJT9,<NA>,<NA>,<NA>,

In [7]:
lab_results_df.describe()

,moisture_content_percent,growth_regulators_ppm,cannabinoid_editor,cannabinoid_d9_thca_percent,cannabinoid_d9_thca_mg_g,cannabinoid_d9_thc_percent,cannabinoid_d9_thc_mg_g,cannabinoid_d8_thc_percent,cannabinoid_d8_thc_mg_g,cannabinoid_cbd_percent,cannabinoid_cbd_mg_g,cannabinoid_cbda_percent,cannabinoid_cbda_mg_g,cannabinoid_cbdv_percent,cannabinoid_cbg_percent,cannabinoid_cbg_mg_g,terpenoid_pinene_percent,terpenoid_pinene_mg_g,microbial_bile_tolerant_cfu_g,microbial_pathogenic_e_coli_cfu_g,microbial_salmonella_cfu_g,mycotoxin_aflatoxins_ppb,mycotoxin_ochratoxin_ppb,metal_arsenic_ppm,metal_cadmium_ppm,metal_lead_ppm,metal_mercury_ppm,pesticide_abamectin_ppm,pesticide_acequinocyl_ppm,pesticide_bifenazate_ppm,pesticide_cyfluthrin_ppm,pesticide_cypermethrin_ppm,pesticide_etoxazole_ppm,pesticide_flonicamid_ppm,pesticide_fludioxonil_ppm,pesticide_imidacloprid_ppm,pesticide_myclobutanil_ppm,pesticide_spinosad_ppm,pesticide_spirotetramet_ppm,pesticide_thiamethoxam_ppm,pesticide_trifloxystrobin_ppm,solvent_butanes_ppm,solvent_heptane_ppm,solvent_propane_ppm,notes,thc_percent,moisture_content_water_activity_rate,solvent_acetone_ppm,solvent_benzene_ppm,solvent_cyclohexane_ppm,solvent_chloroform_ppm,solvent_dichloromethane_ppm,solvent_ethyl_acetate_ppm,solvent_hexanes_ppm,solvent_isopropanol_ppm,solvent_methanol_ppm,solvent_pentanes_ppm,solvent_toluene_ppm,solvent_xylene_ppm,pesticide_acephate_ppm,pesticide_acetamiprid_ppm,pesticide_aldicarb_ppm,pesticide_azoxystrobin_ppm,pesticide_bifenthrin_ppm,pesticide_boscalid_ppm,pesticide_carbaryl_ppm,pesticide_carbofuran_ppm,pesticide_chlorantraniliprole_ppm
count,518034.000000,112274.0,365242.000000,1.930793e+06,1.643794e+06,1.927600e+06,1.646070e+06,136523.000000,3.531700e+04,1.901016e+06,1.595091e+06,1.908667e+06,1.594469e+06,90865.000000,172682.000000,3.979800e+04,17845.000000,15573.000000,5.326270e+05,5.325090e+05,5.002810e+05,6.008430e+05,6.007600e+05,1.848210e+05,184821.000000,184821.000000,1.848210e+05,185059.000000,185050.0,185050.0,185050.0,185050.0,185050.0,185050.0,185050.0,185051.0,185051.0,185050.0,226353.0,185050.0,185050.0,2.442510e+05,2.435180e+05,2.435070e+05,0.0,95627.000000,4.370030e+05,2.444480e+05,2.428330e+05,2.433230e+05,2.432920e+05,2.433140e+05,2.434080e+05,2.436640e+05,2.440990e+05,2.439220e+05,2.434200e+05,2.433110e+05,2.433210e+05,185101.0,185101.0,185101.0,185101.0,185102.000000,185101.0,1.851010e+05,185101.0,185101.0
mean,4.792570,0.0,2880.576172,7.439171e+00,-7.206416e+01,2.373610e+00,-7.659010e+01,-0.211881,-3.963969e+03,1.371000e-01,-8.734264e+01,1.074790e-01,-8.763159e+01,-0.434229,0.194442,-3.516120e+03,-3.722567,0.003067,3.476623e+03,-9.161779e+02,-9.786083e+02,-7.145919e+02,-7.143973e+02,1.082128e+02,0.000013,0.000005,7.196152e-07,10.807364,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-2.517283e+03,-2.583074e+03,-2.622060e+03,NaN,26.966610,-1.258137e+03,-2.563094e+03,-2.594164e+03,-2.587959e+03,-2.589452e+03,-2.588549e+03,-2.584278e+03,-2.584044e+03,-2.564922e+03,-2.575917e+03,-2.585942e+03,-2.588646e+03,-2.587969e+03,0.0,0.0,0.0,0.0,0.000001,0.0,5.672579e-07,0.0,0.0
std,11.632792,0.0,1478.166016,1.766525e+01,2.812215e+04,1.261189e+01,2.810273e+04,17.205599,1.990638e+05,5.610599e+00,2.962581e+04,5.676428e+00,2.963154e+04,20.978998,15.265302,1.875265e+05,62.090500,0.118228,1.370046e+05,9.977886e+04,1.029230e+05,8.844120e+04,8.844731e+04,3.289562e+04,0.002890,0.001778,3.093684e-04,3287.446777,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.605862e+05,1.608250e+05,1.620983e+05,NaN,38.082836,1.121792e+05,1.605431e+05,1.610501e+05,1.608879e+05,1.608981e+05,1.608909e+05,1.608599e+05,1.607754e+05,1.606331e+05,1.606934e+05,1.608559e+05,1.608919e+05,1.608886e+05,0.0,0.0,0.0,0.0,0.000330,0.0,2.440534e-04,0.0,0.0
min,-999.999023,0.0,0.000000,-9.999990e+02,-1.000000e+07,-9.999990e+02,-1.000000e+07,-999.999023,-1.000000e+07,-9.999990e+02,-1.000000e+07,-9.999990e+02,-1.000000e+07,-999.999023,-999.999023,-1.000000e+07,-999.999023,0.000000,-1.000000e+07,-1.000000

In [8]:
# comment out if you are not planning on using Dask
lab_results_df.to_parquet(file_path / 'lab_results.parquet')